### Introdução Camada Gold: 

A camada Gold tem como objetivo organizar os dados em um formato analítico otimizado para consumo por ferramentas de Business Intelligence e análises exploratórias.

Nesta etapa, os dados tratados na camada Silver são modelados segundo os princípios de Data Warehouse, utilizando uma abordagem de modelo dimensional (esquema estrela), no qual uma tabela fato centraliza as métricas do negócio e se relaciona com tabelas dimensão que descrevem os contextos dessas métricas.

Essa modelagem facilita consultas analíticas, melhora a performance de agregações e garante maior clareza semântica para análises e dashboards.

In [0]:
df = spark.table("workspace.games_analytics_silver.games_march_2025")


### Criação das Dimensões: 

Nesta etapa, foram criadas as tabelas dimensão, responsáveis por armazenar atributos descritivos que contextualizam os dados da tabela fato.

Cada dimensão possui uma chave substituta (ID), utilizada para garantir relacionamentos consistentes e evitar dependência direta de campos textuais na tabela fato.

In [0]:
from pyspark.sql import functions as F

dim_publisher = (
    df.select("publisher_normalized")
      .distinct()
      .withColumnRenamed("publisher_normalized", "publisher_name")
      .withColumn("publisher_id", F.monotonically_increasing_id())
)

dim_publisher.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("workspace.games_analytics_gold.dim_publisher")

A dimensão da publisher foi criada a partir do tratamento e modelagem da coluna da publisher na camada silver, trazendo uma centralização e padronização dos dados.

In [0]:
%sql
SELECT * FROM workspace.games_analytics_gold.dim_publisher
LIMIT (10)

In [0]:
from pyspark.sql import functions as F

dim_developer = (
    df.select("developer_normalized")
      .distinct()
      .withColumnRenamed("developer_normalized", "developer_name")
      .withColumn("developer_id", F.monotonically_increasing_id())
)

dim_developer.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("workspace.games_analytics_gold.dim_developer")

A dimensão developer foi criada a partir da padronização da mesma coluna na camada silver, evitando valores duplicados e centralizando os valores.

In [0]:
%sql
SELECT * FROM workspace.games_analytics_gold.dim_developer
LIMIT (10)

In [0]:
from pyspark.sql import functions as F


dim_genre = (
    df.select(F.col("genre_1").alias("genre"))
      .union(df.select(F.col("genre_2").alias("genre")))
      .where(F.col("genre").isNotNull())
      .distinct()
      .withColumn("genre_id", F.monotonically_increasing_id())
)

dim_genre.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("workspace.games_analytics_gold.dim_genre")


A dimensão de gênero consolida os gêneros tratados na camada Silver, permitindo análises categóricas e evitando duplicidade de valores textuais na tabela fato.

In [0]:
%sql
SELECT * FROM workspace.games_analytics_gold.dim_genre
LIMIT (10)

In [0]:
from pyspark.sql import functions as F

dim_time = (
    df.select("release_date")
      .distinct()
      .withColumn("time_id", F.monotonically_increasing_id())
      .withColumn("release_year", F.year("release_date"))
      .withColumn("release_month", F.month("release_date"))
      .withColumn("release_month_name", F.date_format("release_date", "MMMM"))
      .withColumn("release_quarter", F.quarter("release_date"))
      .withColumn("release_semester",
          F.when(F.month("release_date") <= 6, 1).otherwise(2)
      )
)

dim_time.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("workspace.games_analytics_gold.dim_time")


A dimensão de tempo foi criada a partir da data de lançamento dos jogos, permitindo análises por ano, mês e outros recortes temporais.

In [0]:
%sql
SELECT * FROM workspace.games_analytics_gold.dim_time
LIMIT (10)

In [0]:
from pyspark.sql import functions as F

dim_jogo = (
    df.select(
        F.col("name").alias("name")
    )
    .dropDuplicates()
    .withColumn("game_id", F.monotonically_increasing_id())
    .withColumn("created_at", F.current_timestamp())
)

dim_jogo.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("workspace.games_analytics_gold.dim_jogo")

A dimensão do jogo foi criada a partir do tratamento e agrupamento dos jogos por nome na camada silver, evitando duplicatas.

In [0]:
%sql
SELECT * FROM workspace.games_analytics_gold.dim_jogo
LIMIT (10)

In [0]:
fact = (
    df
    # DIM JOGO
    .join(
        dim_jogo,
        df.name == dim_jogo.name,
        "left"
    )

    # DIM PUBLISHER
    .join(
        dim_publisher,
        df.publisher_normalized == dim_publisher.publisher_name,
        "left"
    )

    # DIM DEVELOPER
    .join(
        dim_developer,
        df.developer_normalized == dim_developer.developer_name,
        "left"
    )

    # DIM GENRE (duas vezes)
    .join(
        dim_genre.alias("g1"),
        df.genre_1 == F.col("g1.genre"),
        "left"
    )
    .join(
        dim_genre.alias("g2"),
        df.genre_2 == F.col("g2.genre"),
        "left"
    )

    # DIM TEMPO
    .join(
        dim_time,
        "release_date",
        "left"
    )
)


A tabela fato centraliza as métricas quantitativas relacionadas aos jogos, como avaliações, preços e indicadores derivados, além de armazenar as chaves estrangeiras que referenciam as dimensões criadas anteriormente.

Os relacionamentos foram realizados por meio de joins entre os dados da camada Silver e as dimensões, substituindo atributos textuais por seus respectivos identificadores.

In [0]:
%sql
SELECT * FROM workspace.games_analytics_gold.fact_games
LIMIT (10)

In [0]:
fact_games = fact.select(
    F.col("game_id"),
    F.col("publisher_id"),
    F.col("developer_id"),
    F.col("g1.genre_id").alias("genre_1_id"),
    F.col("g2.genre_id").alias("genre_2_id"),
    F.col("time_id"),

    # métricas
    F.col("price"),
    F.col("approval_pct"),
    F.col("metacritic_score"),
    F.col("positive_ratings_total"),
    F.col("negative_ratings_total"),
    F.col("is_indie"),
    F.col("total_reviews"),
    F.col("windows"),
    F.col("mac"),
    F.col("linux"),
    F.col("approval_weighted"),
    F.col("review_confidence")
)



Após a criação da Fato e o ajuste no relacionamento dela com as tabelas DIM, a partir dos valores vindos na tabela tratada da camada silver, é necessário carregar as métricas de valores diretamente na FATO.

In [0]:
%sql
SELECT * FROM workspace.games_analytics_gold.fact_games
LIMIT (10)

In [0]:
fact_games.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("workspace.games_analytics_gold.fact_games")


Ao final do processo, as tabelas dimensão e a tabela fato foram persistidas na camada Gold, tornando os dados prontos para consumo analítico, criação de views e integração com ferramentas de BI.